In [4]:
import numpy as np
import pandas as pd
import os
import cv2
import ast
from tqdm import tqdm
from PIL import Image, ImageDraw
import pickle
import sys
sys.path.append('../Documentation/Text_Detection')
from CRAFT import CRAFTModel, draw_boxes, draw_polygons, boxes_area, polygons_area
model = CRAFTModel('cuda', use_refiner=True, fp16=True)

/home/icnlab/anaconda3/envs/vllm/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/icnlab/anaconda3/envs/vllm/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/media/icnlab/Data/Manh/OCR/invoice/../Documentation/Text_Detection/CRAFT/craft.py:90: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future re

# Các giá trị config ban đầu

In [5]:
# Đọc dữ liệu
dataset_path = '/media/icnlab/Data/Manh/OCR/invoice/mcocr_train_df.csv'
train_img_path = '/media/icnlab/Data/Manh/OCR/invoice/dataset/train_images'
df = pd.read_csv(dataset_path)
df.head()

,img_id,anno_polygons,anno_texts,anno_labels,anno_num,anno_image_quality
0,mcocr_public_145013ddcph.jpg,"[{'category_id': 15, 'segmentation': [[231.9, ...",MINIMART ANAN|||Chợ Sủi Phú Thị Gia Lâm|||Ngày...,SELLER|||ADDRESS|||TIMESTAMP|||TOTAL_COST|||TO...,5,0.635309
1,mcocr_public_145013fxcgs.jpg,"[{'category_id': 15, 'segmentation': [[311.6, ...",VinCommerce|||VM + QNH Dự án quỹ đất đường sắt...,SELLER|||ADDRESS|||ADDRESS|||ADDRESS|||TIMESTA...,7,0.774317
2,mcocr_public_145013clltn.jpg,"[{'category_id': 15, 'segmentation': [[626.8, ...",SIEU THI BACH HOA TONG HOP|||Bố 5 Cẩm Tây - Cẩ...,SELLER|||ADDRESS|||TOTAL_COST|||TOTAL_COST|||T...,5,0.664084
3,mcocr_public_145013tmibr.jpg,"[{'category_id': 15, 'segmentation': [[715.5, ...",co.op mart|||Co.opMart HAU GIANG|||188 Hau Gia...,SELLER|||SELLER|||ADDRESS|||ADDRESS|||TIMESTAM...,8,0.715504
4,mcocr_public_145013kgypr.jpg,"[{'category_id': 16, 'segmentation': [[200.5, ...","Tổ 7, Khu Minh Tiến A|||VinCommerce|||Ngày bán...",ADDRESS|||SELLER|||TIMESTAMP|||TOTAL_COST|||TO...,5,0.766884


### Tạo list other_class

In [ ]:
# Tính toán IoU 
'''
Tính các giá trị iou giữa các box được cho trong dataframe trên và các box mà CRAFT tạo ra. Nêu giá trị IoU lớn hơn 0.1
thì giữ nguyên các box mà BTC đã label sẵn. Ngược lại nếu nhỏ hơn thì đưa các box đó vào lớp OTHER.
'''
def iou(bbox1: list, bbox2: list) ->float:
    x11, y11, x12, y12 = bbox1
    x21, y21, x22, y22 = bbox2

    area1 = (x12 - x11) * (y12 - y11)
    area2 = (x22 - x21) * (y22 -  y21)

    intersection_width = min(x12, x22) - max(x11, x21)
    intersection_height = min(y12, y22) - max(y11, y21)
    
    if intersection_width <= 0 or intersection_height <= 0:
        return 0

    intersection = intersection_width * intersection_height
    union = area1 + area2 - intersection
    iou = intersection/union 
    return  iou if iou > 0 else 0

# Tạo list other_class
other_class =[]
for i, img_name in enumerate(df['img_id']):
    img_path = os.path.join(train_img_path, img_name)
    img = Image.open(img_path)
    #  Gọi model CRAF
    boxes = model.get_boxes(img)
    annotation_image = {
        'img_name': img_name,
        'bbox':[]
    }
    for box in boxes:
        x1, y1, x2, y2 = box[0][0], box[0][1], box[2][0], box[2][1]
        bbox1 = [x1, y1, x2, y2]
        check = False
        for element in ast.literal_eval(df['anno_polygons'][i]):
            x, y, w, h = element['bbox']
            x1, y1, x2, y2 = x, y, x + w, y + h
            bbox2 = [x1, y1, x2, y2]
            if iou(bbox1,  bbox2) > 0.1:
                check = True
                break

        if check == False:
            annotation_image['bbox'].append(bbox1)
    other_class.append(annotation_image)

# Lưu dữ liệu
with open('box_of_other_class.pkl', 'wb') as f:
    pickle.dump(other_class, f)

## Test
bbox1 = [123, 56, 150, 67]
bbox2 = [123, 56, 150, 67]
print(iou(bbox1, bbox2))

1.0


In [ ]:
# Test CRAFT
import os
img_name = 'mcocr_public_145013ddcph.jpg'
img = Image.open(os.path.join(train_img_path, img_name))
boxes = model.get_boxes(img)

draw = ImageDraw.Draw(img)
for box in boxes:
    draw.rectangle((box[0][0], box[0][1],box[2][0], box[2][1]), outline='red')
img.show() 

type(other_class), other_class[0].keys()

(list, dict_keys(['img_name', 'bbox']))

Opening in existing browser session.


In [ ]:
# #  Kiểm tra lại class other 
# img = Image.open(os.path.join(train_img_path, 'mcocr_public_145013ddcph.jpg'))

# draw = ImageDraw.Draw(img)
# for box in other_class[0]['bbox']:
#     x1, y1, x2, y2 = box
#     draw.rectangle((x1, y1, x2, y2), outline='red')
# img.show()

In [9]:
'''
Sử dụng tập dữ liệu trong dataframe được cho để thành trước 
4 lớp bao gồm SELLER, ADDRESS, TIMESTAMP, TOTAL_COST
'''
df = pd.read_csv(dataset_path)
df.head()


,img_id,anno_polygons,anno_texts,anno_labels,anno_num,anno_image_quality
0,mcocr_public_145013ddcph.jpg,"[{'category_id': 15, 'segmentation': [[231.9, ...",MINIMART ANAN|||Chợ Sủi Phú Thị Gia Lâm|||Ngày...,SELLER|||ADDRESS|||TIMESTAMP|||TOTAL_COST|||TO...,5,0.635309
1,mcocr_public_145013fxcgs.jpg,"[{'category_id': 15, 'segmentation': [[311.6, ...",VinCommerce|||VM + QNH Dự án quỹ đất đường sắt...,SELLER|||ADDRESS|||ADDRESS|||ADDRESS|||TIMESTA...,7,0.774317
2,mcocr_public_145013clltn.jpg,"[{'category_id': 15, 'segmentation': [[626.8, ...",SIEU THI BACH HOA TONG HOP|||Bố 5 Cẩm Tây - Cẩ...,SELLER|||ADDRESS|||TOTAL_COST|||TOTAL_COST|||T...,5,0.664084
3,mcocr_public_145013tmibr.jpg,"[{'category_id': 15, 'segmentation': [[715.5, ...",co.op mart|||Co.opMart HAU GIANG|||188 Hau Gia...,SELLER|||SELLER|||ADDRESS|||ADDRESS|||TIMESTAM...,8,0.715504
4,mcocr_public_145013kgypr.jpg,"[{'category_id': 16, 'segmentation': [[200.5, ...","Tổ 7, Khu Minh Tiến A|||VinCommerce|||Ngày bán...",ADDRESS|||SELLER|||TIMESTAMP|||TOTAL_COST|||TO...,5,0.766884


In [18]:
df['anno_labels'].value_counts()

anno_labels
SELLER|||ADDRESS|||TIMESTAMP|||TOTAL_COST|||TOTAL_COST                                                                        406
SELLER|||ADDRESS|||ADDRESS|||ADDRESS|||TIMESTAMP|||TOTAL_COST|||TOTAL_COST                                                    229
SELLER|||ADDRESS|||ADDRESS|||TIMESTAMP|||TOTAL_COST|||TOTAL_COST                                                               57
SELLER|||ADDRESS|||TIMESTAMP|||TOTAL_COST                                                                                      45
SELLER|||ADDRESS|||ADDRESS|||ADDRESS|||TIMESTAMP|||TIMESTAMP                                                                   37
                                                                                                                             ... 
SELLER|||SELLER|||ADDRESS|||ADDRESS|||TIMESTAMP|||TIMESTAMP|||TIMESTAMP|||TIMESTAMP|||TOTAL_COST|||TOTAL_COST|||TOTAL_COST      1
TOTAL_COST|||TOTAL_COST|||TIMESTAMP|||SELLER|||SELLER|||ADDRESS|||ADDRESS     

In [ ]:
# label = df['anno_labels'][0].split('|||')
# print(label)
# text = df['anno_texts'][0].split('|||')
# print(text)

['SELLER', 'ADDRESS', 'TIMESTAMP', 'TOTAL_COST', 'TOTAL_COST']
['MINIMART ANAN', 'Chợ Sủi Phú Thị Gia Lâm', 'Ngày : 11/08/2020 08:06', 'Tổng tiền:', '74,000']


In [19]:
SELLER = []
ADDRESSS = []
TIMESTAMP = []
TOTAL_COST = []

for i, anno_labels in enumerate(df['anno_labels']):
    if type(anno_labels) == float:
        continue
    anno_labels = anno_labels.split('|||')
    anno_text = df['anno_texts'][i].split('|||')
    for index, label in enumerate(anno_labels):
        if label == 'SELLER':
            SELLER.append(anno_text[index])
        elif label == 'ADDRESS':
            ADDRESSS.append(anno_text[index])
        elif label == 'TIMESTAMP':
            TIMESTAMP.append(anno_text[index])
        elif label == 'TOTAL_COST':
            TOTAL_COST.append(anno_text[index])

In [ ]:
# print(SELLER[:5])
# print(ADDRESSS[:5])
# print(TIMESTAMP[:5])
# print(TOTAL_COST[:5])

['MINIMART ANAN', 'VinCommerce', 'SIEU THI BACH HOA TONG HOP', 'co.op mart', 'Co.opMart HAU GIANG']
['Chợ Sủi Phú Thị Gia Lâm', 'VM + QNH Dự án quỹ đất đường sắt', 'DA QĐ doc tuyến ĐS Tổ 3 Khu 2', 'P.Cẩm Sơn, TP.Cẩm Phả, T.QN', 'Bố 5 Cẩm Tây - Cẩm Phả - QN']
['Ngày : 11/08/2020 08:06', 'Ngày bán: 15/08/2020 11:41', 'Ngày: 21/05/2020', '20 : 42 : 52', 'Ngày bán: 14/08/2020 20:00']
['Tổng tiền:', '74,000', 'TỔNG TIỀN PHẢI T.TOÁN', '30.900', '72.000']


In [25]:
with open('./dataset/text_cls/SELLER_class.pkl', 'wb') as f:
    pickle.dump(SELLER, f)

with open('./dataset/text_cls/ADDRESSS_class.pkl', 'wb') as f:
    pickle.dump(ADDRESSS, f)

with open('./dataset/text_cls/TIMESTAMP_class.pkl', 'wb') as f:
    pickle.dump(TIMESTAMP, f)

with open('./dataset/text_cls/TOTAL_COST_class.pkl', 'wb') as f:
    pickle.dump(TOTAL_COST, f)

In [26]:
'''
Sử dụng thư viện VietOCR để tạo ra class Other cho mô hình phân loại
text
'''
# Import các thư viện cần thiết

from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg

In [32]:
config = Cfg.load_config_from_name('vgg_transformer')
config['weights'] = '../Documentation/Text_Recognize/weights/transformerocr.pth'
config['cnn']['pretrained'] = False
config['device'] = 'cuda:0'
detector = Predictor(config)

In [34]:
img = Image.open('/media/icnlab/Data/Manh/OCR/invoice/dataset/train_images/mcocr_public_145013aagqw.jpg')
img.show()
s = detector.predict(img)
s

'1'

Opening in existing browser session.


In [51]:
# Đọc file bbox của class OTHER
with open('/media/icnlab/Data/Manh/OCR/invoice/dataset/text_detector/box_of_other_class.pkl','rb') as f:
    bbox_other_class = pickle.load(f)

In [53]:
OTHER2

['Giá đăng kỷ :',
 '7,250.000',
 'Gia bán :',
 'HOIP',
 '0,750.000',
 '54, 000.',
 '2910081900000 sữa Thương true milk đường 1',
 'HO P',
 's,000.',
 '20,000',
 'Tổng sổ :',
 '12',
 '74,000.',
 'Tổng cộng :',
 '24, 000.',
 'Tiền khả ch trá.',
 'UND',
 '50,000.',
 'một mì chiếc của Chưa TP.HCM TP.HCM TP.HCM TP.HCM TP.HCM TP.',
 'UND',
 'DN 000 Đ Đức cưa của công.',
 'Mathe!',
 '10!00 đi nh nuôn ơn y nay na',
 'Đêm mua.',
 '176',
 'Điểm hiển thỏi.',
 'Hàng hiện thời.',
 '2.71',
 'Giá trị thưởng tích lũng :',
 'T.Tiền T.Tiền.',
 'cn giá',
 'SL',
 'Xương Úc còn CPP 1 kg.',
 '1551560000000000.00.',
 '30.900.',
 '30.900.',
 'TÔNG TIỂN TÔNG GIÁM',
 '0',
 'TIỂN KHÁCH TRẢ',
 '30.900.',
 'TIỀN M&T',
 '101.000',
 'TIỂN TRÁ LAI',
 '70.100.',
 '(Giá đã bao gồm thuế GTGT)',
 'chỉ xuất hóa đơn trong ngày',
 'Tax invoi ce Wình be issuec Wi thin say',
 'CẢN CN QUỦY KHÁCH VÀ HÀN G&P LAI',
 'Hot) ine : 0247106086600 : V. Ve bsite : viecome',
 'Đến giất.',
 'ĐI TT',
 'ốth.',
 'Tền hắng',
 'BẢ Y TP HCM 31 

In [ ]:
with open('/dataset/text_cls/OTHER_class1.pkl','wb') as f:
    pickle.dump(OTHER1, f)
with open('/dataset/text_cls/OTHER_class2.pkl','wb') as f:
    pickle.dump(OTHER2, f)